In [226]:
def find(s, ch):
    return [i for i, ltr in enumerate(s) if ltr == ch]

def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub)

In [231]:
import csv
import datetime

# initializing the titles and rows list 
fields = [] 
rows = [] 

raw_timestamps_to_hw = dict()
raw_references = dict()
raw_questions_to_hw = dict()

filename = 'contributions.csv'

# reading csv file 
with open(filename, 'r', encoding="utf8") as csvfile: 
    # creating a csv reader object 
    csvreader = csv.reader(csvfile) 
      
    # extracting field names through first row 
    fields = next(csvreader) 
  
    # extracting each data row one by one 
    for row in csvreader: 
        rows.append(row) 

    curr_question = ""
    curr_text = ""
    curr_topic = ""
    curr_folder = ""
    curr_answer = ""
    timestamp = ""
    curr_number = 0
    curr_follow_up = 0
    endorsed = 0
    # This will help us take the most recent version because it always comes first
    already_filled = 0
    
    question_to_num = {}
    num_to_question = {}
    
    for row in rows[:]: 
        # parsing each column of a row 
        for col,field in zip(row, fields): 
            if(field == "Post Number" and curr_number != int(col)):
                curr_folders = curr_folder.split()
                for folder in curr_folders:
                    if folder[-1] == ';':
                            folder = folder[:-1]
                    if timestamp[-3:] == 'UTC':
                        timestamp = timestamp[:-4]
                    date_time_obj = datetime.datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S')
                    if(curr_question != "" and curr_answer != ""):
                        
                        for loc in find_all(curr_answer, "@"):
                            #print(curr_answer)
                            next_space = loc + 1
                            while(next_space <= len(curr_answer)-1 and curr_answer[next_space].isdigit()):
                                next_space = next_space + 1

                            question = -1
                            try:
                                question = int(curr_answer[(loc + 1):next_space])
                            except ValueError:
                                question = -1
                            #print(question)
                            if curr_number in raw_references:
                                raw_references_q = raw_references[curr_number]
                                raw_references_q.append(question)
                                raw_references.update({curr_number : raw_references_q})
                            else:
                                raw_references_q = []
                                raw_references_q.append(question)
                                raw_references.update({curr_number : raw_references_q})
                                
                            if question in raw_references:
                                raw_references_q = raw_references[question]
                                raw_references_q.append(curr_number)
                                raw_references.update({question : raw_references_q})
                            else:
                                raw_references_q = []
                                raw_references_q.append(curr_number)
                                raw_references.update({question : raw_references_q})
                        
                        if folder in raw_timestamps_to_hw:
                            raw_timestamps = raw_timestamps_to_hw[folder]
                            raw_timestamps.append(date_time_obj)
                            raw_timestamps_to_hw.update({folder : raw_timestamps})
                        else:
                            raw_timestamps = []
                            raw_timestamps.append(date_time_obj)
                            raw_timestamps_to_hw.update({folder : raw_timestamps})
                            
                        if folder in raw_questions_to_hw:
                            raw_questions = raw_questions_to_hw[folder]
                            raw_questions.append(curr_question)
                            raw_questions_to_hw.update({folder : raw_questions})
                        else:
                            raw_questions = []
                            raw_questions.append(curr_question)
                            raw_questions_to_hw.update({curr_folder : raw_questions})
                curr_number = int(col)
                curr_follow_up = 0
                already_filled = 0
                curr_answer = ""
                curr_topic = ""
                timestamp = ""
                curr_folder = ""
                endorsed = 0
                
            elif(field == "Submission HTML Removed"): #the question could also be in subject
                curr_text = col
                
            elif(field == "Folders"):
                curr_folder = col
                
            elif(field == "Created At"):
                timestamp = col
                
            elif(field == "Part of Post"):
                if(col == "started_off_question"):
                    if(already_filled == 0):
                        curr_question = curr_text
                    already_filled = 0
                elif(col == "updated_question"):
                    # update question entry to post and follow up numbers
                    if(already_filled == 0):
                        curr_question = curr_text
                        already_filled = 1
                        
                elif(col == "updated_i_answer"):
                    # update last entry to the post and follow up number
                    # what if it was appended? how do we update just the end?
                    if(already_filled == 0):
                        curr_answer = curr_answer + "\n" + curr_text
                        already_filled = 1
                    endorsed = 1
                elif(col == "started_off_i_answer"):
                    if(already_filled == 0):
                        curr_answer = curr_answer + "\n" + curr_text
                    already_filled = 0
                    endorsed = 1
                    
                elif(col == "updated_s_answer"):
                    # update last entry to the post and follow up number
                    # what if it was appended? how do we update just the end?
                    if(already_filled == 0):
                        curr_answer = curr_answer + "\n" + curr_text
                        already_filled = 1
                elif(col == "started_off_s_answer"):
                    # append to answer - if it was previously endorsed and were appending should we keep it endorsed?
                    if(already_filled == 0):
                        curr_answer = curr_answer + "\n" + curr_text
                    already_filled = 0
                    
                elif(col == "followup"):
                    # new entry for the new followup question 
                    
                    curr_folders = curr_folder.split()

                    for loc in find_all(curr_answer, "@"):
                        #print(curr_answer)
                        next_space = loc + 1
                        while(next_space <= len(curr_answer)-1 and curr_answer[next_space].isdigit()):
                            next_space = next_space + 1

                        question = -1
                        try:
                            question = int(curr_answer[(loc + 1):next_space])
                        except ValueError:
                            question = -1
                        #print(question)
                        if curr_number in raw_references:
                            raw_references_q = raw_references[curr_number]
                            raw_references_q.append(question)
                            raw_references.update({curr_number : raw_references_q})
                        else:
                            raw_references_q = []
                            raw_references_q.append(question)
                            raw_references.update({curr_number : raw_references_q})
                            
                        if question in raw_references:
                            raw_references_q = raw_references[question]
                            raw_references_q.append(curr_number)
                            raw_references.update({question : raw_references_q})
                        else:
                            raw_references_q = []
                            raw_references_q.append(curr_number)
                            raw_references.update({question : raw_references_q})
                    
                    for folder in curr_folders:
                        if folder[-1] == ';':
                            folder = folder[:-1]
                        if timestamp[-3:] == 'UTC':
                            timestamp = timestamp[:-4]
                        date_time_obj = datetime.datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S')
                        
                        if folder in raw_timestamps_to_hw:
                            raw_timestamps = raw_timestamps_to_hw[folder]
                            raw_timestamps.append(date_time_obj)
                            raw_timestamps_to_hw.update({folder : raw_timestamps})
                        else:
                            raw_timestamps = []
                            raw_timestamps.append(date_time_obj)
                            raw_timestamps_to_hw.update({folder : raw_timestamps})
                            
                        if folder in raw_questions_to_hw:
                            raw_questions = raw_questions_to_hw[folder]
                            raw_questions.append(curr_question)
                            raw_questions_to_hw.update({folder : raw_questions})
                        else:
                            raw_questions = []
                            raw_questions.append(curr_question)
                            raw_questions_to_hw.update({curr_folder : raw_questions})
                            
                    curr_question = ""
                    curr_answer = ""
                    curr_folder = ""
                    timestamp = ""
                    endorsed = 0
                    already_filled = 0
                        
                    curr_follow_up = curr_follow_up + 1
                    curr_question = curr_text
                    
                elif(col == "reply_to_followup"):
                    curr_answer = curr_answer + "\n" + curr_text

#print(raw_questions_to_hw)

In [232]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

from xml.dom import minidom

folder = "hw2"
print(len(raw_questions_to_hw[folder]))

# parse an xml file by name
mydoc = minidom.parse('Posts.xml')

SEitems = mydoc.getElementsByTagName('row')

SEitems_questions = []
for item in SEitems:
    SEitems_questions.append(item.attributes['Body'].value)
    
SEitems_questions = SEitems_questions[1:6000]

vectorizer = TfidfVectorizer(stop_words='english')
X_orig = vectorizer.fit(SEitems_questions)
X = vectorizer.transform(raw_questions_to_hw[folder])

# Number of clusters per homework
true_k = 50
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=10)
model.fit(X)

# print("\n")
# print("Prediction")



302


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=50, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [233]:
# Combine clusters that have references to each other

Y = vectorizer.transform(["Should we fail for empty queues for delete and contains too?"])
prediction = model.predict(Y)
# print(prediction)

# for x in range(276):
#     if model.labels_[x] == 28:
#         print("Question Number: ") 
#         print(x)
#         print(raw_questions_to_hw[folder][x])

In [234]:
# Create a list of lists for each cluser

final_clusters = {}
for i in range(len(raw_questions_to_hw[folder])):
    label = model.labels_[i]
    final_clusters.setdefault(label, []).append(raw_questions_to_hw[folder][i])
    
print(raw_references)
    
# print(final_clusters)

{22: [92], 92: [22, 98, 190, 3167], 27: [23], 23: [27, 296], 30: [14], 14: [30], 35: [33], 33: [35], 37: [89, 44], 89: [37, 140], 44: [37, 46, 64, 159], 46: [44, 159], 49: [47, 120, 128, 544], 47: [49, 128], 52: [38, 57, 181, 341, 537, 591], 38: [52, 57, 157], 57: [52, 38, 72], 64: [44, 159], 68: [18], 18: [68, 465, 465, 465, 465, 465, 465, 465, 465, 465, 465, 473, 694, 695, 1080, 1132, 1211, 1227], 72: [32, 57, 161], 32: [72], 79: [59], 59: [79], 80: [40], 40: [80], 83: [-1, -1], -1: [83, 83, 99, 99, 108, 123, 123, 135, 172, 213, 251, 283, 283, 310, 324, 327, 477, 478, 499, 594, 594, 790, 883, 923, 923, 976, 986, 1032, 1032, 1032, 1057, 1124, 1158, 1267, 1267, 1285, 1286, 1299, 1299, 1299, 1299, 1299, 1299, 1397, 1405, 1772, 1949, 1982, 2104, 2131, 2138, 2138, 2190, 2190, 2201, 2325, 2332, 2332, 2337, 2412, 2510, 2535, 2617, 2700, 2700, 2757, 2757, 2758, 2758, 2791, 2861, 2945, 2945, 2945, 2945, 2945, 2988, 2999, 3012, 3017, 3031, 3058, 3058, 3058, 3058, 3151, 3203, 3338, 3380, 3399, 

In [235]:
# Shorten the sentences just for one cluster for now
import random
from nltk.tokenize import word_tokenize
import string

number, items = random.choice(list(final_clusters.items()))

stop_words = ["ourselves", "hers", "between", "yourself", "but", "again", 
                  "there", "about", "once", "during", "out", "very", "having", 
                  "with", "they", "own", "an", "be", "some", "for", "do", "its", 
                  "yours", "such", "into", "of", "most", "itself", "other", "off",
                  "is", "s", "am", "or", "who", "as", "from", "him", "each", "the",
                  "themselves", "until", "below", "are", "we", "these", "your", "his", 
                  "through", "don", "nor", "me", "were", "her", "more", "himself", "this",
                  "down", "should", "our", "their", "while", "above", "both", "up", "to",
                  "ours", "had", "she", "all", "no", "when", "at", "any", "before", "them",
                  "same", "and", "been", "have", "in", "will", "on", "does", "yourselves", 
                  "then", "that", "because", "what", "over", "why", "so", "can", "did", 
                  "not", "now", "under", "he", "you", "herself", "has", "just",
                  "where", "too", "only", "myself", "which", "those", "i", "after",
                  "few", "whom", "t", "being", "if", "theirs", "my", "against", "a",
                  "by", "doing", "it", "how", "further", "was", "here", "than", "ok",
                  "thank", "ah"]

length = len(items)
shortened_questions = []
for i in range(0, length):
    question = items[i]
    tokens = word_tokenize(question)
    tokens = [w.lower() for w in tokens]
    words = [w for w in tokens if (not w in stop_words and w != "" and w != " ")]

    # Get rid of punctuation
    table = str.maketrans('', '', string.punctuation)
    words = [w.translate(table) for w in words]

    final_question = " ".join(words)

    shortened_questions.append(final_question)

#     shortened_questions = [s for s in shortened_questions if s != '']

#     # Printing out average
# totalNumWords = 0
# maxLength = 0
# for s in shortened_questions:
#     numWords = len(word_tokenize(s))
#     if (maxLength < numWords): 
#         maxLength = numWords
#     # print(numWords)
#     if (numWords > 50):
#         print(s)
#         print("\n")
#     totalNumWords += numWords

print(shortened_questions)



['ta s coming office hours today ', 'office hour queue todays office hours mcclelland ']


In [236]:
from bert_serving.client import BertClient 

# Start BERT client for mrpc bert-serving-start -model_dir /Users/maralevy/Downloads/uncased_L-12_H-768_A-12 -tuned_model_dir /Users/maralevy/Documents/Piazza-SeniorDesign/bert/mrpc_output -num_worker=4 -max_seq_len=50
# Start BERT client for qqa checkpoint bert-serving-start -model_dir /Users/maralevy/Downloads/uncased_L-12_H-768_A-12 -tuned_model_dir /Users/maralevy/Documents/Piazza-SeniorDesign/bert/qqp_output -num_worker=4 -max_seq_len=50 -ckpt_name bert_model.ckpt-1000
bc = BertClient(check_version=False)  
doc_vecs = bc.encode(shortened_questions)

In [237]:
# This cell allows you to type a question and find semantically similar questions using BURT, eventually
# we should run this for every question in raw_questions

import np
print(doc_vecs)
topk = len(shortened_questions)
final_scores = {}
for question in shortened_questions:
    query_vec = bc.encode([question])
    # compute simple dot product as score
    score = np.sum(query_vec * doc_vecs, axis=1)/np.linalg.norm(doc_vecs, axis=1)
    topk_idx = np.argsort(score)[::-1][:topk]
#     print("Question: ")
#     print(question)
    for idx in topk_idx:
        if idx != topk_idx[0]:
            final_scores.setdefault(items[topk_idx[0]], []).append((score[idx], items[idx]))

print(final_scores)

[[-0.89218897  0.09297748  0.31572938 ... -0.5483665  -0.6428468
   0.6810323 ]
 [-0.9384423   0.6642193   0.0735179  ...  0.09992716 -0.6974508
   0.39765537]]
{"Are the TA's coming to office hours today?": [(15.133161, 'Is there an office hour queue for todays office hours in McClelland?')], 'Is there an office hour queue for todays office hours in McClelland?': [(14.489926, "Are the TA's coming to office hours today?")]}


In [238]:
cluster_number = 0
final_clusters = {}
used_strings = {}
for key in final_scores:
    quota = float(final_scores[final_scores[key][0][1]][0][0] - final_scores[final_scores[key][0][1]][1][0])
    if key == final_scores[final_scores[key][0][1]][0][1] or \
    (key == final_scores[final_scores[key][0][1]][1][1] and \
     (abs(final_scores[final_scores[key][0][1]][1][0] - final_scores[final_scores[key][0][1]][0][0]) < quota * 2)) or \
    (key == final_scores[final_scores[key][0][1]][2][1] and \
     (abs(final_scores[final_scores[key][0][1]][2][0] - final_scores[final_scores[key][0][1]][0][0]) < quota * 3)):        
        # Combine clusters if needed
        if key in used_strings and final_scores[key][0][1] in used_strings:
            cluster_number_one = used_strings[key]
            cluster_number_two = used_strings[final_scores[key][0][1]]
            if cluster_number_two != cluster_number_one:
                final_clusters[cluster_number_one] = final_clusters[cluster_number_one] + final_clusters[cluster_number_two]
                for string in final_clusters[cluster_number_two]:
                    used_strings[string] = cluster_number_one
                final_clusters.pop(cluster_number_two, None)
        elif key in used_strings:
            final_clusters[used_strings[key]].append(final_scores[key][0][1])
        elif final_scores[key][0][1] in used_strings:
            final_clusters[used_strings[final_scores[key][0][1]]].append(key)
        else:
            used_strings[key] = cluster_number
            used_strings[final_scores[key][0][1]] = cluster_number
            final_clusters[cluster_number] = [key, final_scores[key][0][1]]
            cluster_number = cluster_number + 1
            
for key in final_clusters:
    print("Cluster %d:"% key)
    for q in final_clusters[key]:
        print("Q: \n %s" % q)
        


IndexError: list index out of range